In [ ]:
# === Imports ===
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from tqdm import tqdm

# === Params ===
DATA_CSV = Path("./price.csv")                 # 旧: /Users/ohno/... を廃止
OUT_ROOT = Path("./images/stride/60day-win")            # 出力のルート
SPAN = 60                                           # 表示日数（提案法は60日）
# TAUS = [1, 3, 10]                                   # τの候補
TAUS = [10]                                   # τの候補


RSI_PERIOD = 14                                     # RSIの期間
FIG_DPI = 100
FIGSIZE_INCH = (224/FIG_DPI, 224/FIG_DPI)           # 224x224px相当（後段でResizeするなら任意）

# (N−1)τ=30 → Nを事前に持っておくと便利
N_for_tau = {tau: (30 // tau) + 1 for tau in TAUS}  # {1:31, 3:11, 10:4}
N_for_tau


{10: 4}

In [2]:
# === Load price.csv ===
df = pd.read_csv(DATA_CSV, index_col='Date', parse_dates=True)
df = df.sort_index()             # 念のため日付昇順に整列
df = df[['Open','High','Low','Close'] + [c for c in ['SMA5','SMA25','RSI'] if c in df.columns]]

# === テクニカルが無ければこのセルで作る（dataset.ipynb と整合） ===
need_sma5  = 'SMA5'  not in df.columns
need_sma25 = 'SMA25' not in df.columns
need_rsi   = 'RSI'   not in df.columns

if need_sma5:
    df['SMA5'] = df['Close'].rolling(window=5, min_periods=5).mean()
if need_sma25:
    df['SMA25'] = df['Close'].rolling(window=25, min_periods=25).mean()

if need_rsi:
    # RSI(14): 上昇分/下落分を分けて14SMAし、RSI = 100 * up / (up + down)
    diff = df['Close'].diff()
    up   = diff.clip(lower=0.0)
    down = (-diff).clip(lower=0.0)
    up_sma   = up.rolling(RSI_PERIOD, min_periods=RSI_PERIOD).mean()
    down_sma = down.rolling(RSI_PERIOD, min_periods=RSI_PERIOD).mean()
    df['RSI'] = 100.0 * (up_sma / (up_sma + down_sma))

df = df.dropna().copy()  # 描画に必要な列の欠損を落としてから使う
print(df.shape, df.columns.tolist())


(4961, 7) ['Open', 'High', 'Low', 'Close', 'SMA5', 'SMA25', 'RSI']


In [3]:
def save_candle_60d(df_slice, out_path):
    # 念のため：必要な列とDatetimeIndexをチェック
    assert isinstance(df_slice.index, pd.DatetimeIndex)
    for col in ['Open','High','Low','Close','SMA5','SMA25','RSI']:
        assert col in df_slice.columns, f"missing column: {col}"

    add_plots = [
        mpf.make_addplot(df_slice['SMA5'],  panel=0, color='orange',  width=0.8),
        mpf.make_addplot(df_slice['SMA25'], panel=0, color='blue',    width=0.8),
        mpf.make_addplot(df_slice['RSI'],   panel=1, color='black',   width=0.8),
    ]

    mpf.plot(
        df_slice,
        type="candle",
        style="classic",
        figsize=FIGSIZE_INCH,      # (幅, 高さ)インチ
        tight_layout=True,
        axisoff=True,              # 軸非表示（環境によっては未対応。次の修正Bを参照）
        addplot=add_plots,
        savefig={'fname': str(out_path), 'dpi': FIG_DPI}  # ← ここでdpiを指定
    )


In [ ]:
OUT_ROOT.mkdir(parents=True, exist_ok=True)
dates = df.index.strftime('%Y-%m-%d').to_list()
num = len(df)

for tau in TAUS:
    N = N_for_tau[tau]                   # 1→31, 3→11, 10→4
    out_tau = OUT_ROOT / f"{tau}stride"
    out_tau.mkdir(parents=True, exist_ok=True)
    print(f"[tau={tau}] N={N}  ->  out={out_tau}")

    # i は「セットの開始インデックス」
    # 最後の画像の終端が num-1 を越えないように、 i の上限を調整
    #   最後の画像の終端 = i + (N-1)*tau + (SPAN-1)  <= num-1
    #   i <= num - SPAN - (N-1)*tau
    i_max = num - SPAN - (N-1)*tau
    for i in tqdm(range(i_max), desc=f"tau={tau}"):
        # セット全体のフォルダ名（例：2018-01-05~2018-04-19）
        set_start = dates[i]
        set_end   = dates[i + (N-1)*tau + (SPAN-1)]
        set_dir = out_tau / f"{set_start}~{set_end}"
        set_dir.mkdir(parents=True, exist_ok=True)

        # k=0, tau, 2*tau, ... で N枚の60日画像を作る
        for k in range(0, (N-1)*tau + 1, tau):
            s = i + k                # 開始
            e = s + SPAN             # 終了はスライスの終端（非含む）
            window = df.iloc[s:e]    # ilocで位置スライス（60行のはず）
            # 例：tau=10, N=4 のとき:
            #  k=0  : [i           , i+60)
            #  k=10 : [i+10        , i+70)
            #  k=20 : [i+20        , i+80)
            #  k=30 : [i+30        , i+90)

            # ファイル名に k をゼロ埋めして「並び順」を保証
            fname = f"{k:02d}_{dates[s]}~{dates[e-1]}.png"
            save_candle_60d(window, set_dir / fname)


[tau=10] N=4  ->  out=images\stride\60day\10stride


tau=10:  97%|█████████▋| 4705/4871 [37:35<01:33,  1.78it/s]  

In [ ]:
# 例: tau=10 の最初のセットの枚数を確認
check_dir = next((OUT_ROOT / "10stride").glob("*"))
pngs = sorted(check_dir.glob("*.png"))
print(check_dir, len(pngs), "files ->", [p.name for p in pngs[:4]])
